# Group Assignment

1. We can define the second difference matrix as follows ($N=5$ for example):
   $$
    A = \begin{pmatrix}
    2 &-1  &  & &\\
     -1 & 2 &-1  & \\
     & -1 & 2 &-1  \\
     & & -1  &2 &-1 \\
     & & & -1 & 2
    \end{pmatrix}
    $$
    a) Let the true solution $x_a$ be generated by the normal distributed random numbers, thus $b = Ax$ 
    b) The convergence condition can be set as $||x_{n+1} − x_n|| \leq \epsilon$ for some small value of $\epsilon$. Choose your own $\epsilon$, try to modify the sample script `jacobi_method.ipynb` to count how many loop iterations occur before convergence. How does it change with the change of $N$? 
    c) The weighted Jacobi method uses a parameter $\omega$ to compute the iteration as
   $$
    \mathbf {x} ^{(k+1)}=\left(1-\omega \right)\mathbf {x} ^{(k)}+ \omega\mathbf {x} ^{(k+1)}_{\text{Jacobi}}=\left(1-\omega \right)\mathbf {x} ^{(k)}+ \omega D^{-1}(\mathbf {b} -(L+U)\mathbf {x} ^{(k)})
   $$
    with $\omega =2/3$ being the usual choice. Modify your script to implement the weighted Jacobi method. Does it converge faster than the Jacobi method?
    Hint: You can read *Jacobi_method_Wikipedia.pdf* for your reference. 

2. Apply the Gauss-Seidel method to solve the linear equations given in Problem 1.
   a)  Choose your own $\epsilon$, try to modify the sample script `gauss_seidel_method.ipynb` to count the executing time before convergence. How does it change with the change of $N$? 
   b) Does Gauss-Seidel method converge, as expected, faster than the Jacobi method?
   Hint: You can read *Gauss_Seidel_method_Wikipedia.pdf* for your reference. 

3. The method of successive over-relaxation (SOR) is a variant of the Gauss–Seidel method for solving a linear system of equations, resulting in faster convergence, where the iterative is defined as
   $$
   \begin{aligned}
   \mathbf{x}^{(k+1)}&= (1-\omega)\mathbf{x}^{(k)}+\omega \mathbf{x}^{(k+1)}_{\text{Gauss-Seidel}}\\
   x_i^{(k+1)}&=(1-\omega)x_i^{(k)}+\frac{\omega}{a_{ii}}\left(b_i-\displaystyle\sum_{j<i}a_{ij}x_j^{(k+1)}-\displaystyle\sum_{j>i}a_{ij}x_j^{(k)}\right)
   \end{aligned}
   $$
   where $0<\omega <2$. $\omega >1$ is called the relaxation factor. 
   Consider the linear system as follows:
   $$
   \begin{cases}
   3x_1 - x_2+ x_3 = -1\\
   -x_1 +3x_2 -x_3 = 7\\
   x_1 - x_2 +3x_3 = -7\\
      \end{cases}
   $$
   a) Set your own $\epsilon$, try to modify the sample script `gauss_seidel_method.ipynb` to implement the SOR method.
   b) Can you find the optimal value of $\omega$ empirically? Please compare your result with the theoretical prediction - the optimal relaxation parameter $\omega$ is given by
    $$
    \omega_\text{opt} = 1+\left(\frac{\lambda_J}{1+\sqrt{1-\lambda_J^2}}\right)^2 
    $$
    $\lambda_J$ is the spectral radius calculated  in the Jacobi method.   
    Hint: You can read *SOR_method_Wikipedia.pdf* for your reference. 
   

4. One application of the conjugate gradient method is to solve the normal equation to find the least-square solution of an (over-determined) equation system $\mathbf{A}\mathbf{x}=\mathbf{b}$, where the coefficient matrix $\mathbf{A}$ is $M$ by $N$ of rank $n$, i.e., $M\geq N=n$.The normal equation of this system is 
$$
\mathbf{A}^{T}\mathbf{A}\mathbf{x}=\mathbf{A}^{T}\mathbf{b}% 
$$
Here $\mathbf{A}^{T}\mathbf{A}$ is an $N$ by $N$ symmetric, positive definite matrix. When $A$ is a large-sized sparse symmetrical matrix, conjugate gradient method is also preferred. 
Modify the sample script  `conj_grad.ipynb` to solve Problem 1. Please compare your results with the results from the Jacobi method and the Cholesky method. 

5. Read Example 7.2[^1] and the sample script `power_method_eig.ipynb`. The problem is to find the largest eigenvalue of matrix $A$:
   $$
   A = \begin{pmatrix}
   4 & -i & 2 \\
   i & 2 & 2+7i \\
   2 & 2-7i & -2
   \end{pmatrix}
   $$
  a) Try modifying the above example to count how many loop iterations occur before convergence. Is it larger or smaller than you had expected?  
  b) An alternative to using the Rayleigh quotient to determine whether convergence is achieved is the convergence criterion 
   $$
   ||x_{n+1} − x_n|| \leq \epsilon
   $$
    This is especially useful if we do not need to compute the eigenvalue, and can save some computational time. Modify the code above to use this convergence criterion, and compare the overall wall time to the original version.

6. We can use the normalised power method above on $A^{−1}$ to calculate $\lambda_N$ — the smallest magnitude eigenvalue of $A$, and the dominant eigenvalue of $A^{−1}$. In practice, however, matrix inversion is a difficult numerical computation, is prone to error, and can be quite unstable. A more common approach than direct matrix inversion is to view the inverse power iteration as a system of linear equations to be solved involving $A$:  
Step 1. Choose a normalised starting vector $x_0$ where $||x_0|| = 1$;   
Step 2. Solve the system of linear equations $Ax_{n+1} = x_n$ to determine the next vector in the iteration, and normalise this vector;
Step 3. Repeat step 2 until convergence is achieved, $||x_{n+1} − x_n|| \leq \epsilon$ for some small value of $\epsilon$. The Rayleigh quotient $x_n^\dagger Ax_n$ will provide the value of the smallest magnitude eigenvector of $A$.    
Modify the sample script to find the smallest eigenvalue of matrix $A$.

[^1]: Joshua Izaac, Jingbo Wang, Computational Quantum Mechanics, Springer, 2018